1. get geo index for each county
2. write function to get DataFrame of 1. city 2. type 3. page token (optional)
3. write function to get 60 places DataFrame
4. get multiple city / types
5. join them
6. profit!

In [1]:
# Dependencies
import requests
import json
import time
import gmaps
import pandas as pd
import numpy as np
import pprint as pprint
# Google API Key
from config import gkey

def to_lat_lng(json):
  return '{0}, {1}'.format(json["lat"], json['lng'])

def get_cities_df(cities):
    points = []
    north_easts = []
    south_wests = []
    
    for target_city in cities: 
        # Build the endpoint URL
        target_url = ('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(target_city, gkey)
        # Run a request to endpoint and convert result to json
        response = requests.get(target_url).json()
        geometry = response["results"][0]["geometry"]

        # Extract latitude and longitude.
        points.append(to_lat_lng(geometry["location"]))
        north_easts.append(to_lat_lng(geometry["bounds"]["northeast"]))
        south_wests.append(to_lat_lng(geometry["bounds"]["southwest"]))

    return pd.DataFrame({
        "City": cities,
        "Point": points,
        "Northeast": north_easts,
        "Southwest": south_wests,
    }) 

# cities = ["Palo Alto, California"]
cities = ["San Francisco, California", "Redwood City, California", "Berkeley, California", "Palo Alto, California", "Mountain View, California", "Sunnyvale, California", "Cupertino, California", "Santa Clara, California", "San Jose, California", "Fremont, California"]
# cities_df = get_cities_df(cities)
# cities_df.to_csv('cities.csv')



In [2]:


def get_response(url, params):
    response = requests.get(url, params=params).json()
    if response['status'] == 'OK':
        return response
    else:
        print('Not OK response: ' + response['status'])
        return None

# Returns DataFrame
def request_places(city, location, type, gkey):    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # rankby:distance will result INVALID_REQUSET
    params = {
        "location": location,
        "radius": 10000, # hard-code distance
        "type": type,
        "key": gkey,
    }

    response = get_response(base_url, params)
    results= response['results']
    count = 1
    
    # If there is no page token, return result, otherwise concat result with next page.
    while (count < 3 and 'next_page_token' in response):
        count = count + 1
        token = response['next_page_token']
#         print(f'Getting next page: {count}, token: {token}')
        
        # Sleep a while so that token can be used.
        time.sleep(3)
        params = {
            "location": location,
            "radius": 10000, # hard-code distance
            "type": type,
            "key": gkey,
            "page_token": response['next_page_token'],
        }
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = get_response(base_url, params)
        results.extend(response['results'])
    
    print(f'Total page count: {count}')
    return results

def to_data_frame(city, type, results):
    cities = []
    types = []
    place_ids = []
    names = []
    poi = []
    vicnities = []
#     locations = []
    lat = []
    lng = []
    
    for result in results:
        # Skip locality place, e.g. Palo Alto.
        if ('locality' in result['types']):
            continue

        cities.append(city)
        types.append(type)
        place_ids.append(result['place_id'])
        names.append(result['name'])
        poi.append(result['types'])
        vicnities.append(result['vicinity'])
#         locations.append(to_lat_lng(result['geometry']['location']))
        lat.append(result['geometry']['location']['lat'])
        lng.append(result['geometry']['location']['lng'])
    
    return pd.DataFrame({
        'City': cities,
        'Type': types,
        'Place ID': place_ids,
        'Name': names,
        'Point of interests': poi,
        'Address': vicnities,
#         'LatLng': locations,
        'Lat': lat,
        'Lng': lng,
    })

def get_places_data_frame(city, location, type, gkey):
    print(f'Getting plcaes, city: "{city}", location: "{location}", type: "{type}"');
    results = request_places(city, location, type, gkey)
    return to_data_frame(city, type, results) 

In [3]:
cities_df = pd.read_csv("cities.csv")
type = 'local_government_office'

def get_all_places_data_frame(type, gkey):
    frames = [];
    for index, row in cities_df.iterrows():
        frames.append(get_places_data_frame(row['City'], row['Point'], type, gkey))
    return pd.concat(frames)

place_df = get_all_places_data_frame(type, gkey)
place_df.to_csv(type + '.csv')
place_df





Getting plcaes, city: "San Francisco, California", location: "37.7749295, -122.4194155", type: "local_government_office"
Total page count: 3
Getting plcaes, city: "Redwood City, California", location: "37.48521520000001, -122.2363548", type: "local_government_office"
Total page count: 3
Getting plcaes, city: "Berkeley, California", location: "37.8715926, -122.272747", type: "local_government_office"
Total page count: 3
Getting plcaes, city: "Palo Alto, California", location: "37.4418834, -122.1430195", type: "local_government_office"
Total page count: 3
Getting plcaes, city: "Mountain View, California", location: "37.3860517, -122.0838511", type: "local_government_office"
Total page count: 3
Getting plcaes, city: "Sunnyvale, California", location: "37.36883, -122.0363496", type: "local_government_office"
Total page count: 3
Getting plcaes, city: "Cupertino, California", location: "37.3229978, -122.0321823", type: "local_government_office"
Total page count: 3
Getting plcaes, city: "Sant

,City,Type,Place ID,Name,Point of interests,Address,Lat,Lng
0,"San Francisco, California",local_government_office,ChIJlbvCgJ6AhYARv4JSejr6lU8,San Francisco Municipal Transportation Agency,"[local_government_office, point_of_interest, e...","1 South Van Ness Avenue, San Francisco",37.774955,-122.419055
1,"San Francisco, California",local_government_office,ChIJd2spnJuAhYARbbDmxMLr-ws,Project Read-Main Library,"[library, local_government_office, point_of_in...","100 Larkin Street, San Francisco",37.779116,-122.415766
2,"San Francisco, California",local_government_office,ChIJAe-oMNd_j4AR-PSfbqF0BbY,California Unemployment Insurance Appeals Board,"[local_government_office, point_of_interest, e...","185 Berry Street # 200, San Francisco",37.776507,-122.392039
3,"San Francisco, California",local_government_office,ChIJf21VOpqAhYARIgY0yWCWuP8,Supreme Court of California,"[courthouse, local_government_office, point_of...","350 McAllister Street, San Francisco",37.780734,-122.417853
4,"San Francisco, California",local_government_office,ChIJjzgIo56AhYARtXdJAWnHwt0,Public Works Department Engineering,"[local_government_office, point_of_interest, e...","30 Van Ness Avenue # 5100, San Francisco",37.775970,-122.419055
5,"San Francisco, California",local_government_office,ChIJp2UYiWKAhYARWbFNX3-c7bA,Victorian Government Business Office,"[local_government_office, finance, point_of_in...","575 Market Street, San Francisco",37.789660,-122.400436
6,"San Francisco, California",local_government_office,ChIJF-l7ZYiAhYARU-zELCWxKwA,VisaCentral San Francisco,"[local_government_office, point_of_interest, e...","555 Montgomery Street, San Francisco",37.794371,-122.403610
7,"San Francisco, California",local_government_office,ChIJEfL_42SAhYARDf1dtbmQT68,The State Bar of California,"[lawyer, local_government_office, point_of_int...","180 Howard Street, San Francisco",37.791159,-122.393274
8,"San Francisco, California",local_government_office,ChIJ3UTZGJqAhYARWnUjlHYTJOk,Assemblyman Tom Ammiano,"[local_government_office, point_of_interest, e...","455 Golden Gate Avenue # 14300, San Francisco",37.781114,-122.418062
9,"San Francisco, California",local_government_office,ChIJzwpUCZyAhYARpD64Jc7SNb4,San Francisco City & County: Department of Chi...,"[local_government_office, point_of_interest, e...","1390 Market Street #900, San Francisco",37.777084,-122.417681


In [6]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)
# Store 'Lat' and 'Lng' into  locations 
locations_2 = place_df[["Lat", "Lng"]].astype(float)

In [7]:
# Create bank symbol layer
company_layer = gmaps.symbol_layer(
    locations_2, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
)


fig = gmaps.figure()
fig.add_layer(company_layer)

fig

Figure(layout=FigureLayout(height='420px'))